In [1]:
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_runner
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import os
import itertools

fast_mode = (fls.env=='local')
importlib.reload(fls)

MainProcess
MainProcess


<module 'flg_support' from 'd:\\flagellar/code/core\\flg_support.py'>

In [2]:
# Prepare datasets
if not fast_mode:
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models/', fls.result_dir + '/many_full/')
    fls.download_kaggle_dataset('jeroencottaar/byu-many-models-abbreviated/', fls.result_dir + '/many_abbr/')

In [ ]:
#fls.profiling=True
importlib.reload(flg_runner)
import git 
repo = git.Repo(search_parent_directories=True)
git_commit_id = repo.head.object.hexsha
for i in itertools.count():
    r = flg_runner.baseline_runner(fast_mode=fast_mode)
    r.git_commit_id = git_commit_id
    r.seed = i
    #r.base_model.run_in_parallel = False
    base_filename = r.label + '_' + str(r.seed) + '_' + git_commit_id[:8]
    output_file_full = fls.result_dir + '/many_full/' + base_filename + '_f.pickle'
    output_file_abbr = fls.result_dir + '/many_abbr/' + base_filename + '_a.pickle'
    if os.path.isfile(output_file_full):
        continue
    r.run()
    fls.dill_save(output_file_full, r)
    r.trained_model = 0
    fls.dill_save(output_file_abbr, r)
    if not fast_mode:
        fls.upload_kaggle_dataset(fls.result_dir + '/many_full/')
        fls.upload_kaggle_dataset(fls.result_dir + '/many_abbr/')

444 200
10 5
{'seed': 1, 'scale_percentile_value': 3.047286498801027, 'img_size': 576, 'model_name': 'yolo11m', 'use_pretrained_weights': False, 'box_size': 29, 'trust': 5, 'fix_norm_bug': False, 'weight_decay': 0.00042332644897257566, 'hsv_h': 0.015, 'hsv_s': 0.0, 'hsv_v': 0.0, 'translate': 0.0, 'scale': 0.5274591760723646, 'fliplr': 0.5, 'flipud': 0.5, 'degrees': 0, 'shear': 0.0, 'mosaic': 0.0, 'mixup': 0.0, 'erasing': 0.0, 'use_albumentations': False, 'confidence_threshold': 0.5000729345260105}
Timer unit: 1e-07 s

Total time: 0.43698 s
File: d:\flagellar/code/core\flg_preprocess.py
Function: load_and_preprocess at line 40

Line #      Hits         Time  Per Hit   % Time  Line Contents
    40                                               @fls.profile_each_line
    41                                               def load_and_preprocess(self, data, desired_original_slices = None):
    42                                           
    43         1    2505326.0    3e+06     57.3       